# Assumptions

1. Only account owners are important, account users play little part in my analysis
2. Date of Analysis assumed to be December 31, 1999. Right before the end of the world
3. Assumed ordinal credit card types i.e. junior$<$classic$<$gold
4. Percentage of total money paid for loan matches percentage of actual payments made
5. Put 0 in for NaNs in columns A12, A15. Assumed this was reasonable given the data dictionary

# Imports

In [201]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import datetime

In [202]:
loan_df = pd.read_csv("data/loan_info.csv")
account_df = pd.read_csv('data/account_info.csv')
payment_df = pd.read_csv('data/payment_info.csv')

# Altering the Loans table

In [203]:
pd.set_option("display.max_columns", None)

In [204]:
loan_df.head()

,account_id,owner_client_id,owner_disp_id,gender,num_acct_users,owner_birth_date,statement_freq,card_type,card_issue_date,acct_start_date,loan_id,loan_date,loan_amt,total_paid,loan_duration_mos,total_payments,monthly_payments,loan_status,district_id,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,2,2,2,M,2,1945-02-04,POPLATEK MESICNE,NaN,NaN,1993-02-26,4959,1994-01-05,80952,80952.0,24,24,3373.0,A,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107
1,19,25,25,F,1,1939-04-23,POPLATEK MESICNE,NaN,NaN,1995-04-07,4961,1996-04-29,30276,20184.0,12,8,2523.0,B,21,Tabor,south Bohemia,103347,87,16,7,1,7,67.0,9104,1.5,2.07,123,2299.0,2354
2,25,31,31,M,1,1962-02-09,POPLATEK MESICNE,NaN,NaN,1996-07-28,4962,1997-12-08,30276,30276.0,12,12,2523.0,A,68,Frydek - Mistek,north Moravia,228848,15,40,18,2,6,57.2,9893,4.0,4.72,96,5623.0,5887
3,37,45,45,M,1,1952-08-26,POPLATEK MESICNE,NaN,NaN,1997-08-18,4967,1998-10-14,318480,5308.0,60,1,5308.0,D,20,Strakonice,south Bohemia,70646,94,14,3,1,4,58.4,8547,2.6,3.64,120,1563.0,1542
4,38,46,46,F,1,1940-01-30,POPLATEK TYDNE,NaN,NaN,1997-08-08,4968,1998-04-19,110736,18456.0,48,8,2307.0,C,19,Prachatice,south Bohemia,51428,50,11,3,1,4,52.7,8402,3.1,3.98,120,999.0,1099


In [205]:
loan_df['statement_freq'].value_counts()

POPLATEK MESICNE      559
POPLATEK TYDNE         91
POPLATEK PO OBRATU     32
Name: statement_freq, dtype: int64

In [206]:
loan_df.dtypes

account_id             int64
owner_client_id        int64
owner_disp_id          int64
gender                object
num_acct_users         int64
owner_birth_date      object
statement_freq        object
card_type             object
card_issue_date       object
acct_start_date       object
loan_id                int64
loan_date             object
loan_amt               int64
total_paid           float64
loan_duration_mos      int64
total_payments         int64
monthly_payments     float64
loan_status           object
district_id            int64
A2                    object
A3                    object
A4                     int64
A5                     int64
A6                     int64
A7                     int64
A8                     int64
A9                     int64
A10                  float64
A11                    int64
A12                  float64
A13                  float64
A14                    int64
A15                  float64
A16                    int64
dtype: object

### Altering the date formats

In [207]:
# Convert our datetime columns to actual datetimes
loan_df['owner_birth_date'] = pd.to_datetime(loan_df['owner_birth_date'], format='%Y-%m-%d')
loan_df['acct_start_date'] = pd.to_datetime(loan_df['acct_start_date'], format='%Y-%m-%d')
loan_df['card_issue_date'] = pd.to_datetime(loan_df['card_issue_date'], format='%Y-%m-%d')

In [208]:
# A datetime object is not useful for analysis, so let's make the birthdates an age in years
loan_df['owner_age'] = ((datetime.datetime(1999,12,31)-loan_df['owner_birth_date']).dt.days/365.25).astype(int)

# Credit card length measured in months, since credit cards are billed monthly
loan_df['mos_with_card'] = np.floor((datetime.datetime(1999,12,31)-loan_df['card_issue_date']).dt.days/12)

# We will do months for Account length as well, since again, most accounts are billed monthly. 
# (some weekly and per transaction, but keep it monthly)
loan_df['account_len_mos'] = ((datetime.datetime(1999,12,31)-loan_df['acct_start_date']).dt.days/12).astype(int)

### Mapping Data to new types

In [209]:
loan_df.card_type.unique()

array([nan, 'classic', 'gold', 'junior'], dtype=object)

In [210]:
# Map the card types to numbers
card_map = {np.nan:0,'junior':1,'classic':2,'gold':3}
loan_df['card_type']=loan_df['card_type'].map(card_map)

In [211]:
# Make Gender boolean
# use a single column
gender_map = {'M':1,'F':0}
loan_df['is_male'] = loan_df['gender'].map(gender_map)

In [212]:
# Change loan_status to successful/failed, ie. boolean
# A/C True
# B/D False

# This will be our TARGET VARIABLE

status_map = {'A':1, 'B':0, 'C':1, 'D':0}
loan_df['target'] = loan_df['loan_status'].map(status_map)

### Dummy some data

In [213]:
# Whatever this is, too many uniques, its a string, so let's just drop it
loan_df.A2.nunique()

77

In [214]:
# Whatever this is, a decent amount of uniques, let's dummy it
loan_df.A3.nunique()

8

In [215]:
# Add on dummied states (I guess is what they are?)
# I.E. Column A3
loan_df = pd.concat([loan_df,pd.get_dummies(loan_df.A3)],axis=1)

In [216]:
# Dummy the statement frequency
# first map the Czech names to english
loan_df['statement_freq'].value_counts()
freq_map = {'POPLATEK MESICNE':'monthly_acct_bill',
            'POPLATEK TYDNE':'weekly_acct_bill',
            'POPLATEK PO OBRATU':'trans_acct_bill'}
loan_df['statement_freq'] = loan_df['statement_freq'].map(freq_map)

loan_df = pd.concat([loan_df,pd.get_dummies(loan_df.statement_freq)], axis=1)

In [217]:
loan_df.head()

,account_id,owner_client_id,owner_disp_id,gender,num_acct_users,owner_birth_date,statement_freq,card_type,card_issue_date,acct_start_date,loan_id,loan_date,loan_amt,total_paid,loan_duration_mos,total_payments,monthly_payments,loan_status,district_id,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,owner_age,mos_with_card,account_len_mos,is_male,target,Prague,central Bohemia,east Bohemia,north Bohemia,north Moravia,south Bohemia,south Moravia,west Bohemia,monthly_acct_bill,trans_acct_bill,weekly_acct_bill
0,2,2,2,M,2,1945-02-04,monthly_acct_bill,0,NaT,1993-02-26,4959,1994-01-05,80952,80952.0,24,24,3373.0,A,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107,54,NaN,208,1,1,1,0,0,0,0,0,0,0,1,0,0
1,19,25,25,F,1,1939-04-23,monthly_acct_bill,0,NaT,1995-04-07,4961,1996-04-29,30276,20184.0,12,8,2523.0,B,21,Tabor,south Bohemia,103347,87,16,7,1,7,67.0,9104,1.5,2.07,123,2299.0,2354,60,NaN,144,0,0,0,0,0,0,0,1,0,0,1,0,0
2,25,31,31,M,1,1962-02-09,monthly_acct_bill,0,NaT,1996-07-28,4962,1997-12-08,30276,30276.0,12,12,2523.0,A,68,Frydek - Mistek,north Moravia,228848,15,40,18,2,6,57.2,9893,4.0,4.72,96,5623.0,5887,37,NaN,104,1,1,0,0,0,0,1,0,0,0,1,0,0
3,37,45,45,M,1,1952-08-26,monthly_acct_bill,0,NaT,1997-08-18,4967,1998-10-14,318480,5308.0,60,1,5308.0,D,20,Strakonice,south Bohemia,70646,94,14,3,1,4,58.4,8547,2.6,3.64,120,1563.0,1542,47,NaN,72,1,0,0,0,0,0,0,1,0,0,1,0,0
4,38,46,46,F,1,1940-01-30,weekly_acct_bill,0,NaT,1997-08-08,4968,1998-04-19,110736,18456.0,48,8,2307.0,C,19,Prachatice,south Bohemia,51428,50,11,3,1,4,52.7,8402,3.1,3.98,120,999.0,1099,59,NaN,72,0,1,0,0,0,0,0,1,0,0,0,0,1


### Numeric Alterations

Next, calculate the total paid/loan_amt for the percentage paid

Assume here that the percentage paid will match the percentage of total payments made (this would be a doable but annoying check since the percentages will be inherently different because of the different scaling of the features, so I'll stick with this assumption in the interest of time)

In [218]:
loan_df['percent_paid'] = (loan_df['total_paid']/loan_df['loan_amt']).round(2)

### Fill NaNs with 0s

In [219]:
loan_df.isna().nunique()

account_id           1
owner_client_id      1
owner_disp_id        1
gender               1
num_acct_users       1
owner_birth_date     1
statement_freq       1
card_type            1
card_issue_date      2
acct_start_date      1
loan_id              1
loan_date            1
loan_amt             1
total_paid           1
loan_duration_mos    1
total_payments       1
monthly_payments     1
loan_status          1
district_id          1
A2                   1
A3                   1
A4                   1
A5                   1
A6                   1
A7                   1
A8                   1
A9                   1
A10                  1
A11                  1
A12                  2
A13                  1
A14                  1
A15                  2
A16                  1
owner_age            1
mos_with_card        2
account_len_mos      1
is_male              1
target               1
Prague               1
central Bohemia      1
east Bohemia         1
north Bohemia        1
north Morav

In [220]:
loan_df = loan_df.fillna(0)

### Drop Columns

In [221]:
# These columns will not be useful in the analysis
# loan_id doesn't help with anything, its just a different number for every record
# same with disp_id and client_id.
# Loan date is not useful, since loan_duration captures all relevant information
# Dates used how they were needed
# A2 not useful
# A3 dummied
# statement_freq dummied
# gender made boolean elsewhere
# loan_status used as target, made boolean elsewhere
# total_paid and total_payments info included through percent_paid

loan_df.drop(columns=['owner_disp_id', 
                      'loan_id', 
                      'owner_client_id', 
                      'loan_date',
                      'owner_birth_date',
                      'statement_freq',
                      'card_issue_date',
                      'acct_start_date',
                      'A2',
                      'A3',
                      'gender',
                      'loan_status',
                      'total_paid',
                      'total_payments'
                     ], inplace=True)

# Payments Table

In [222]:
payment_df.head()

,account_id,k_symbol,count(trans_id),sum(amount)
0,1,SIPO,40,98080.0
1,2,SIPO,65,472290.0
2,2,UVER,24,80952.0
3,3,POJISTNE,13,46007.0
4,3,SIPO,13,14755.0


In [223]:
new = pd.pivot_table(payment_df, 
                     index=['account_id'], 
                     columns=['k_symbol'], 
                     aggfunc='max'
                    ).reset_index()

In [224]:
new.head()

account_id count(trans_id)             sum(amount)                   
k_symbol                   POJISTNE  SIPO  UVER    POJISTNE      SIPO     UVER
0                 1             NaN  40.0   NaN         NaN   98080.0      NaN
1                 2             NaN  65.0  24.0         NaN  472290.0  80952.0
2                 3            13.0  13.0   NaN     46007.0   14755.0      NaN
3                 4             NaN  29.0   NaN         NaN   60262.0      NaN
4                 5             NaN  14.0   NaN         NaN   37352.0      NaN

In [225]:
new.columns = new.columns.map(''.join)

In [226]:
new.head()

,account_id,count(trans_id)POJISTNE,count(trans_id)SIPO,count(trans_id)UVER,sum(amount)POJISTNE,sum(amount)SIPO,sum(amount)UVER
0,1,NaN,40.0,NaN,NaN,98080.0,NaN
1,2,NaN,65.0,24.0,NaN,472290.0,80952.0
2,3,13.0,13.0,NaN,46007.0,14755.0,NaN
3,4,NaN,29.0,NaN,NaN,60262.0,NaN
4,5,NaN,14.0,NaN,NaN,37352.0,NaN


In [227]:
new.columns

Index(['account_id', 'count(trans_id)POJISTNE', 'count(trans_id)SIPO',
       'count(trans_id)UVER', 'sum(amount)POJISTNE', 'sum(amount)SIPO',
       'sum(amount)UVER'],
      dtype='object')

In [228]:
col_map = {'account_id':'account_id', 
           'count(trans_id)POJISTNE':'num_insurance_transactions',
           'count(trans_id)SIPO':'num_household_transactions',
           'count(trans_id)UVER':'num_loan_transactions', 
           'sum(amount)POJISTNE':'total_insurance_amt_paid',
           'sum(amount)SIPO':'total_house_amt_paid',
           'sum(amount)UVER':'total_loan_amt_paid'}

In [229]:
payment_alter = new.rename(columns=col_map).fillna(0)

In [230]:
# We do not need loan info in this table because I plan to join it with the loan table
payment_alter.drop(columns=['num_loan_transactions', 'total_loan_amt_paid'], inplace=True)

## Merge Payments and Loan

In [231]:
loan_df = loan_df.merge(payment_alter, 
                        how='inner',
                        on='account_id',
                        left_index = True
                       )

In [233]:
loan_df.tail()

,account_id,num_acct_users,card_type,loan_amt,loan_duration_mos,monthly_payments,district_id,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,owner_age,mos_with_card,account_len_mos,is_male,target,Prague,central Bohemia,east Bohemia,north Bohemia,north Moravia,south Bohemia,south Moravia,west Bohemia,monthly_acct_bill,trans_acct_bill,weekly_acct_bill,percent_paid,num_insurance_transactions,num_household_transactions,total_insurance_amt_paid,total_house_amt_paid
3506,11327,1,0,39168,24,1632.0,7,94725,38,28,1,3,6,63.4,9920,2.2,2.87,130,4289.0,4846,20,0.0,67,0,1,0,1,0,0,0,0,0,0,1,0,0,0.12,0.0,9.0,0.0,33237.0
3507,11328,1,0,280440,60,4674.0,54,387570,0,0,0,1,1,100.0,9897,1.6,1.96,140,18721.0,18696,47,0.0,95,0,1,0,0,0,0,0,0,1,0,1,0,0,0.08,0.0,0.0,0.0,0.0
3508,11349,2,0,419880,60,6998.0,1,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107,54,0.0,140,0,1,1,0,0,0,0,0,0,0,0,0,1,0.63,0.0,39.0,0.0,424554.0
3509,11359,1,2,54024,12,4502.0,61,117897,139,28,5,1,6,53.8,8814,4.7,5.74,107,2112.0,2059,31,138.0,159,1,1,0,0,0,0,0,0,1,0,1,0,0,1.00,0.0,46.0,0.0,207828.0
3510,11362,1,0,129408,24,5392.0,67,106054,38,25,6,2,6,63.1,8110,5.7,6.55,109,3244.0,3079,37,0.0,128,0,1,0,0,0,0,1,0,0,0,1,0,0,1.00,34.0,34.0,11220.0,162520.0


In [234]:
loan_df.to_csv('data/loan_analysis_data.csv',index=False)